## Summary Data for GraphML Files

In [1]:
import graph_tool.all as gt
import os
import numpy as np
import graph_tool
from tqdm import tqdm_notebook

from lib.analysis import clustering
from lib.analysis import dist
from lib.analysis import model
from lib.analysis import power
from lib.analysis import stats

In [2]:
data = 'data'
ignore = ['.DS_Store', 'us']
graphs = {}

In [3]:
for country in [d for d in os.listdir(data) if d not in ignore]:
    path = os.path.join(data, country)
    for file in [f for f in os.listdir(path) if f not in ignore]:
        if file.endswith('.graphml'):
            fpath = os.path.join(path, file)
            graphs[file[:-8]] = gt.load_graph(fpath)

In [4]:
with open("summary.csv", "w+") as f:
    f.write(','.join(['Number of Edges', 'Number of Vertices',
                      'Clustering Coefficient', 'Average Degree',
                      'Average Excess Degree', 'Average Distance',
                      'Size of LCC', 'Number of CC']) + '\n')
    for name, g in graphs.items():
        print(name)
        f.write(f'{name},{str(stats.summary(g))}\n')

net_se
net_fr_se
net_fr_an
net_ch_cs
net_ch_cn
uk
net_dk
net_at
